# Expected vs. Realized Move in Options Using Random Forest Regression

This educational project investigates the relationship between **expected** and **realized moves** ($EM$ and $RM$, respectively) in equity options, focusing on the challenge of forecasting how much an underlying asset will move based on option market signals.

Using a **Random Forest regression** approach—a common machine learning method—we examine how closely the **market’s expectations** ($EM$), reflected in an option contract’s price and its implied volatility, correspond to the **actual realized price movements** ($RM$). We then build a Random Forest Regressor to predict the magnitude of $RM$ for a fixed horizon ($h$) and analyze the conditions under which these forecasts provide meaningful insight.

This project aims to deepen our understanding of how predictive market-implied volatility is for future movements, advancing our knowledge of risk, price forecasting, and volatility management in financial decision-making and equity investing.


## Introduction to Stock Market Trading, Options & $EM$ vs. $RM$

Stock prices change continually as investors buy and sell shares. This movement is influenced by company news, economic data, and trader sentiment, leading to periods of rising (bull) or falling (bear) prices.
 
**Options** are financial contracts that give buyers the right, but not the obligation, to buy or sell an underlying stock at a specific price (the **strike price**, $K$) on or before a set date (the **expiry**). In other words, options let you bet on whether a stock’s price will go up or down, and at what level you can make that bet.
- **Call options:** Gives the right to **buy** the stock at $K$. This is like betting that the stock price will go above a certain level. The payoff formula for a call option at expiry is: $$\max(S_T - K, 0)$$
Where,
    - $K$ is the **strike price**—the predetermined price at which the option holder can buy the stock.
    - $S_T$ is the **stock price at the time of expiry** ($T$).
    - The expression $(S_T - K)$ represents how much the stock price is above the strike price at expiry.
    - The $\max(\cdot, 0)$ function means you take the greater of $(S_T - K)$ or 0. This guarantees that if the option ends up "out-of-the-money" (i.e., $S_T < K$), the payoff cannot be negative. The option contract then will simply expires worthless.

     Put simply: if you have a call option, you get paid the difference if the stock ends up higher than your agreed price, otherwise, you get nothing. You’d only use your option if you could buy cheaper than in the market. If not, you just let it expire and lose nothing extra.


- **Put options:** Gives the right to **sell** the stock at $K$. This is like betting that the stock price will fall below a certain level. The payoff formula for a put option at expiry is: $$\max(K - S_T, 0)$$
Where,
    - $K$ is the **strike price**—the predetermined price at which the option holder can sell the stock.
    - $S_T$ is the **stock price at the time of expiry** ($T$).
    - The expression $(K - S_T)$ represents how much the strike price is above the stock price at expiry.
    - The $\max(\cdot, 0)$ function means you take the greater of $(K - S_T)$ or 0. This ensures that if the option finishes "out-of-the-money" (i.e., $S_T > K$), the payoff cannot be negative. The put then simply expires worthless.

     Put simply (no pun intended): if you have a put option, you get paid the difference if the stock ends up lower than your agreed sell price, otherwise, you get nothing. You’d only use your option if you could sell higher than in the market. If not, you just let it expire and lose nothing extra.
 
Option prices are influenced not just by the current stock price, but also by the expected amount the stock will move by expiry. This expectation is captured in the **implied volatility (IV)** of the option. This is a measure derived from option prices that reflects the market's consensus on how much the stock might fluctuate.

Traders often use IV to estimate the **Expected Move ($EM$)**, which determines how much the stock price is likely to change over a given period. The **expected move** over a horizon of $h$ days (using at-the-money implied volatility) is calculated as follows:

$$
\text{EM}_{t,h} = \sigma^{ATM}_t \sqrt{\frac{h}{252}}
$$

Where:
- $\text{EM}_{t,h}$ is the **Expected Move** over the next $h$ trading days, starting from time $t$.
- $\sigma^{ATM}_t$ is the **at-the-money implied volatility** at time $t$ (usually quoted as an annualized percentage, such as 20%).
- $h$ is the **number of trading days** in your forecast window (for example, $h=5$ for a week, $h=21$ for a month).
- 252 is the **typical number of trading days in a year** (used to annualize or de-annualize volatility).
- The square root component, $\sqrt{\frac{h}{252}}$, adjusts the annualized implied volatility to the shorter time frame of $h$ days. This reflects the idea from financial theory that volatility scales with the square root of time.

In plain language:
- The formula takes the market’s current “best guess” (from option prices) for how much the stock would fluctuate in a year, then converts it—using the square-root rule—to estimate the likely move over just $h$ days.
- For example, if you want to know how much the stock might move in the next week, plug in $h=5$.
- This gives you a market-implied estimate of the range within which the stock price might end up after $h$ days.

After the period ends, we can observe the **Realized Move ($RM$)**, which is the actual movement that occurred in the underlying stock over the horizon we’re measuring. The formula for realized move over $h$ days is:

$$\text{RM}_{t,h} = \left| \ln \left( \frac{P_{t+h}}{P_t} \right) \right|$$

Where,
- $\text{RM}_{t,h}$ is the **Realized Move** between time $t$ and $t+h$ (the *actual* observed change over $h$ days).
- $P_t$ is the **stock price at the start** (time $t$).
- $P_{t+h}$ is the **stock price at the end** of the $h$-day period.
- The fraction $\frac{P_{t+h}}{P_t}$ tells us how much the price has changed relative to its starting value.
- The natural logarithm $\ln(\cdot)$ is used to measure the size of the price change in a way that is fair for both upward and downward moves, and lets us easily compare changes across different prices or stocks.
- The absolute value $|\cdot|$ ensures that we are measuring the magnitude of the movement, regardless of whether the stock price went up or down.

In plain language:
- The realized move is the absolute size of the log-return from $t$ to $t+h$. It shows how much the stock actually changed in price (up or down) over the period, without regard to direction, just the size.

Suppose the stock price today is 100 USD. The options market is implying a 5% expected move ($EM$) over the next week. In this scenario, people expect the price could end up anywhere between $95 and $105. This is written as:
$$
\text{EM}_{t,5\text{D}} = 5%
$$

Two possible outcomes can occur by the end of the week:

* **If the stock finishes at $108:**
  The realized move ($RM$) is
  $$
  \text{RM} = \left| \ln\left( \frac{108}{100} \right) \right| \approx 7.7% \approx 8%
  $$
  This is larger than what the options market predicted, meaning the stock was more volatile than expected.

* **If the stock finishes at $103:**
  The realized move ($RM$) is
  $$
  \text{RM} = \left| \ln\left( \frac{103}{100} \right) \right| \approx 3%
  $$
  This is smaller than the expected move; the stock was calmer than the options market anticipated.

In this example, intuitively, if the market priced in a $5 move but the stock ran up $8, that indicates unexpected volatility; if it only moved $3, things were quieter than expected.

If you calculated the realized move not just from the starting and ending prices but based on the **highest and lowest** prices during the week, you’d get a larger number because this method captures all the swings up and down, not just where you started and finished.

Finally, if you look at realized moves over longer horizons (like a month instead of a week), the big ups and downs can cancel each other out or get averaged in, which typically smooths out the realized move.




## What is Regression? What is Random Forest Regression?



**Regression** is a type of machine learning that predicts a continuous numeric value, given a set of features. In our case, we use regression to forecast the realized move ($RM$, actual volatility) of a stock, using data such as option prices, historical price patterns, and more.

A **Random Forest** is an ensemble learning method that builds many decision trees and averages their predictions. Here’s how it works:

- Each **decision tree** is trained on a random sample of the data (with replacement — aka bootstrapping).
- At each split in a tree, only a random subset of features is considered, making every tree see a slightly different part of the data.
- The **final prediction** is the average (for regression) of all trees’ predictions.

For a Random Forest, the prediction for an input $X$ is calculated as:
$$
\hat{y} = \frac{1}{N} \sum_{i=1}^{N} T_i(X)
$$
Where,

- $\hat{y}$ is the final prediction made by the Random Forest.
- $N$ represents the total number of decision trees in the forest.
- $T_i(X)$ is the prediction for $X$ made by the $i$-th decision tree.
- The sum $\sum_{i=1}^{N} T_i(X)$ means we add up the predictions from all the trees.
- Finally, we divide by $N$ to take the average, so the Random Forest output is the mean of all the tree predictions.

This process helps the Random Forest provide more stable and accurate predictions by pooling together the results from many different trees.

**We can use a Random Forest for this problem because:**
- It can model highly **nonlinear relationships**—which are common in finance.
- It’s highly effective for **tabular data** with many features (like ours).
- It’s **robust to outliers and multicollinearity** (when features are correlated).
- By averaging many trees, it reduces overfitting and variance (the **bias–variance trade-off**) without much need for feature scaling or heavy preprocessing.

Random Forest also allows tuning parameters to improve performance:
- `n_estimators`: Number of trees (more = smoother, more robust predictions)
- `max_depth`, `max_features`: Control tree complexity and diversity
- `min_samples_split`, `min_samples_leaf`: Prevent trees from growing too fine (“overfitting” small blips)

Here is a visual example of a Random Forest Regression (sourced from https://www.spotfire.com/glossary/what-is-a-random-forest):
<img src="random-forest-diagram.svg" alt="Random Forest Diagram" style="width:40%; display:block; margin:auto;">
<em>
Figure Above: A Random Forest is an ensemble of decision trees, each trained on a bootstrapped sample of the dataset and a random subset of features. Each tree makes its own prediction independently, capturing different aspects of the data’s underlying patterns. The forest then aggregates these individual outputs, averaging for regression tasks to produce a more stable, accurate, and less overfitted final prediction.
</em>


The goal of this project is to model and predict $RM$ (the realized, or actual, volatility) using features including option market data, with a **Random Forest Regression** model.




## Introduction to the Datasets

We are modeling equity-level options for a small, diverse trading universe: AAPL (Apple), MSFT (Microsoft), NVDA (Nvidia), TSLA (Tesla), AMZN (Amazon), JPM (JPMorgan) and XOM (Exxon Mobil).

To predict the realized move ($RM$) in equity options, we will need datasets that include the open, high, low, close, and volume (OHLCV) data, along with option chain data for the equities in our trading universe.


We are going to want to install and import the following packages in our project:
- `yfinance` : The primary API for fetching both the OHLCV and option-chain data
- `pandas` : Core data manipulation
- `numpy` : Numerical operations (log returns, volatility math)
- `tqdm` : Progress bars when looping through tickers or dates
- `python-dateutil` : Date parsing, shifting horizons
- `os` : File operations to store and organize downloaded or processed data

In [1]:
%pip install yfinance pandas numpy tqdm python-dateutil 


Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
import os
print("Packages imported successfully")

Packages imported successfully


From `yfinance`, we are going to pull the following datasets:

- **Equity OHLCV Data**:

    Variables: `Open`, `High`, `Low`, `Close`, `adjClose`, `volume_OHLCV`

    Sampling: Daily frequency spanning 5+ years (e.g, 2018-2024)

    With this data, we are going to:

    - Compute log returns
    - Compute realized move, such that $RM_{t,h} = |\ln(P_{t+h}/P_t)|$
    - Generate rolling volatility, momentum, and volume-based features

- **Option-Chain Data**:

    Variables (for both calls and puts): `strike`, `lastPrice`, `impliedVolatility`, `volume_OptionsChain`, `openInterest`, `inTheMoney`

    Frequency: On-demand snapshot (current option-chain at the time of data pull) 

    With this data, we are going to:

    - Extract at-the-money Implied Volatility (ATM IV) as a proxy for expected volatility
    - Compute expected move, suchs that $EM_{t,h} = \sigma_{ATM,t} \sqrt{h/252}$
    - Track IV over time by saving daily snapshots for historical reconstruction.

    **NOTE**: Historical option IV data is NOT directly available from yfinance. Therefore, we capture daily ATM IV values and store them in snapshot files (e.g. `/Snapshots_IV/iv_snapshot_YYYY-MM-DD.csv`) to construct an evolving IV time series. This approach enables consistent training and evaluation as more data accumulates.


Our data integration pipeline is visualized as follows:

``` vbnet
yfinance OHLCV ──┐
                  ├──> Merge on (ticker, date) → Feature Engineering → Model Dataset
yfinance Option Chains ─┘
```

- We collect the data via downlaoding daily OHLCV and option chain data for all tickers.
- For each ticker, we find the strike price nearest to the current stock price — our at-the-money (ATM) strike. 
    - At this ATM strike price, there exists both a call option and put option, with their own implied volatility. We take the average of the call IV and put IV as shown: $$ \text{ATM IV} = \frac{\text{IV}(\text{call}) + \text{IV}(\text{put})}{2}$$
    - We store this in the variable `atm_iv`.
- We calculate the expected move (EM) with $EM_{t,h} = atm_iv \times \sqrt{h/252}$.
- We create a target with the realized move (RM) with $RM_{t,h} = |\ln(P_{t+h}/P_t)|$.
- We then merge `ticker` and `date`, ensuring all features use past information relative to realized mmove (RM).
- We then clean the data by handling missing IVs, winorize outliers and drop non-trading days to prepare for feature engineering


We first instantiate the configurations for this data:

In [ ]:
# Define trading universe & time period
TICKERS = ['AAPL', 'MSFT', 'NVDA', 'TSLA', 'AMZN', 'JPM', 'XOM']
END = pd.Timestamp.today().normalize()
START = END - relativedelta(years=7) # Comfortably exceeds the 5+ years regimen  

# Store data in directories & confirm directories exist
DATA_DIR = './Data'
OHLCV_DIR = os.path.join(DATA_DIR, 'Snapshots_OHLCV')
IV_DIR = os.path.join(DATA_DIR, 'Snapshots_IV')
os.makedirs(OHLCV_DIR, exist_ok=True)
os.makedirs(IV_DIR, exist_ok=True)

print("Configurations applied. Successful run.")

Configurations applied. Successful run.


We’ll then define small helper functions to make this project’s code more modular and maintainable, providing a set of reusable functions for data cleaning and other utility tasks. 

1. `_standardize_ohlcv_columns(df)`
* Renames columns (e.g., `Open → open`, `Close → close`) and ensures only the standard OHLCV columns are kept.
* Keeps consistent naming and numeric formats across tickers.

2. `_clean_ohlcv(df)`
* Handles missing or non-trading days, ensures only business days remain, forward-fills small data gaps, and removes zero-volume rows.
* Produces a clean, continuous daily time series suitable for modeling.

3. `_winsorize_series(s, lower_q=0.01, upper_q=0.99)`
* Clips the extreme 1% of values on both ends of a numeric series.
* Reduces the impact of outliers (for example, unrealistic implied-volatility spikes).

4. `_nearest(items, value)`
* Finds the element in a list (e.g., strikes) closest to a target value (e.g., current stock price).
* Lets the script select the ATM (at-the-money) option when extracting implied volatility.



In [14]:
def  _standardize_ohlcv_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize OHLCV column names and numeric values, ensuring only standard columns are kept.
    
    Args:
        df (pd.DataFrame): Input DataFrame with OHLCV columns
    Returns:
        pd.DataFrame: DataFrame with standardized column names
    """

    # Renaming columns to standard format
    rename_hash = {
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Adj Close': 'adjClose',
        'Volume': 'volume_OHLCV'
    }
    df = df.rename(columns=rename_hash)
    keep = ["open", "high", "low", "close", "adjClose", "volume_OHLCV"]
    df = df[[c for c in keep if c in df.columns]]

    # Convert columns to numeric values. If no conversion is possible, set to NaN value using errors="coerce"
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


def _clean_ohlcv(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans an OHLCV DataFrame by removing gaps, non-business days, and rows with missing or zero volume.

    Args:
        df (pd.DataFrame): DataFrame with OHLCV data indexed by date

    Returns:
        pd.DataFrame: Cleaned, continuous business-day OHLCV DataFrame
    """

    # Ensure index is DatetimeIndex, drop missing dates, and sort in order
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index, errors="coerce")
    df = df[~df.index.isna()]
    df = df.sort_index()

    # Keep only business days and rows that actually traded (volume > 0 if present)
    bdays = pd.bdate_range(df.index.min(), df.index.max())
    df = df.reindex(bdays) # Align to business days; will introduce NAs on gaps
    
    # Drop rows where all price fields are NA (non-trading or missing)
    price_cols = [c for c in ["open", "high", "low", "close", "adj_close"] if c in df.columns]
    df = df.dropna(axis=0, how="all", subset=price_cols)

    # If volume exists, remove rows with zero volume (common for non-trading placeholders)
    if "volume_OHLCV" in df.columns:
        df = df[(df["volume_OHLCV"].isna()) | (df["volume_OHLCV"] > 0)]

    # Forward-fill then back-fill minor data gaps in price fields only
    df[price_cols] = df[price_cols].ffill().bfill()

    # Final NA drop if anything remains critical
    df = df.dropna(subset=price_cols, how="any")
    return df


def _winsorize_series(s: pd.Series, lower_q: float = 0.01, upper_q: float = 0.99) -> pd.Series:
    """
    Clips the values in a numeric series at the lower and upper quantiles (lower_q and upper_q), to reduce the impact of extreme outliers.

    Args:
        s (pd.Series): Numeric series to winsorize
        lower_q (float, optional): Lower quantile threshold. Defaults to 0.01.
        upper_q (float, optional): Upper quantile threshold. Defaults to 0.99.

    Returns:
        pd.Series: Winsorized series with outliers capped at the specified quantiles
    """

    # Return the series unchanged if it is empty after dropping NaN values
    if s.dropna().empty:
        return s
    
    # Clip values at lower and upper quantiles to cap outliers
    lower_bound = s.quantile(lower_q)
    upper_bound = s.quantile(upper_q)
    return s.clip(lower_bound, upper_bound)


def _nearest(items, value):
    """
    Finds the element in a list (e.g., strikes) closest to a target value (e.g., current stock price).

    Args:
        items (list): List of items to search through   
        value (float): Target value to find nearest to

    Returns:
        object: Nearest item from the list
    """
    
    # Find the element in items that is numerically closest to value, ignoring NaNs
    items = np.asarray(items, dtype=float)
    idx = np.nanargmin(np.abs(items - float(value)))
    return items[idx]

print("Helper functions defined successfully.")

Helper functions defined successfully.
